# Importing libs

In [2]:
import pandas as pd
import numpy as np
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import udf, avg, col
from snowflake.snowpark.types import IntegerType, FloatType, StringType, BooleanType
from snowflake.snowpark.files import SnowflakeFile
import sys
import logging
sys.path.append('..')
from credentials import Credentials

In [3]:
cred = Credentials()
session = Session.builder.configs(cred.__dict__).create()

In [172]:
session.use_role("SYSADMIN")
session.use_database("ACCOUNTADMIN_MGMT")
session.use_warehouse("ACCOUNTADMIN_MGMT")
session.use_schema("UTILITIES")

# Working with individual table

## Individual table

In [173]:
# Getting name of the table
db_name = 'STITCH'
schema_name = 'SALESFORCEQASIT'
table_name:str = 'CASE'

In [174]:
def transform_name(name, type, name_alias):
    if isinstance(type, str) and 'VARCHAR' in type:
        #return name + '0007'
        return f"CAST(SUBSTR({name},1,2000) AS VARCHAR(2000)) AS {name_alias}".format(name, name_alias)
    else:
        return f"{name} AS {name_alias}".format(name, name_alias)

In [175]:
table_view_mapping = session.sql(f"SELECT * FROM ACCOUNTADMIN_MGMT.UTILITIES.SALESFORCE_TABLE_VIEW_MAPPING WHERE NAME_SALESFORCE_ENVIRONMENT = '{schema_name}' AND NAME_SALESFORCE_TABLE_ORIGINAL = '{table_name}'".format(schema_name, table_name)).collect()
table_view_mapping_dic = [row.asDict() for row in table_view_mapping][0]
table_name = table_view_mapping_dic['NAME_SALESFORCE_TABLE_ORIGINAL']
table_for_desc_name = table_view_mapping_dic['NAME_SNOWFLAKE_RESERVED_WORD']
view_name = table_view_mapping_dic['NAME_SALESFORCE_VIEW_ALIAS']

In [176]:
result_describe_table = [row.as_dict() for row in session.sql(f"DESCRIBE TABLE {db_name}.{schema_name}.{table_for_desc_name};".format(db_name, schema_name, table_for_desc_name)).collect()]

In [177]:
df_describe_table = pd.DataFrame(result_describe_table)
df_describe_table.drop(columns=['null?', 'default','primary key', 'unique key',
'check', 'expression', 'comment', 'policy name'], inplace=True)

In [178]:
result_salesforce_mapping = [row.as_dict() for row in session.sql(f"SELECT NAME_SALESFORCE_ATTRIBUTE, NAME_ALIAS FROM ACCOUNTADMIN_MGMT.UTILITIES.SALESFORCE_COLUMN_MAPPING WHERE NAME_SALESFORCE_OBJECT = '{table_name}'".format()).collect()]
df_salesforce_mapping = pd.DataFrame(result_salesforce_mapping)

In [179]:
if df_salesforce_mapping.empty:
	final_df = df_describe_table
	final_df['NAME_ALIAS'] = final_df['name']
else:
	final_df = pd.merge(df_describe_table, df_salesforce_mapping, left_on='name', right_on='NAME_SALESFORCE_ATTRIBUTE', how='left')
	final_df['NAME_ALIAS'] = final_df.apply(lambda x: x['name'] if pd.isna(x['NAME_ALIAS']) else x['NAME_ALIAS'], axis=1)

In [180]:
final_df['name_new'] = final_df.apply(lambda x: transform_name(x['name'], x['type'], x['NAME_ALIAS']), axis=1)

In [181]:
final_df

,name,type,kind,NAME_SALESFORCE_ATTRIBUTE,NAME_ALIAS,name_new
0,ACCOUNTID,VARCHAR(16777216),COLUMN,ACCOUNTID,ACCOUNTID,"CAST(SUBSTR(ACCOUNTID,1,2000) AS VARCHAR(2000)..."
1,ACCOUNT_EXECUTIVE1__C,VARCHAR(16777216),COLUMN,ACCOUNT_EXECUTIVE1__C,ACCOUNT_EXECUTIVE1__C,"CAST(SUBSTR(ACCOUNT_EXECUTIVE1__C,1,2000) AS V..."
2,ACCOUNT_EXECUTIVE__C,VARCHAR(16777216),COLUMN,ACCOUNT_EXECUTIVE__C,ACCOUNT_EXECUTIVE__C,"CAST(SUBSTR(ACCOUNT_EXECUTIVE__C,1,2000) AS VA..."
3,ACCOUNT_ID__C,VARCHAR(16777216),COLUMN,ACCOUNT_ID__C,ACCOUNT_ID__C,"CAST(SUBSTR(ACCOUNT_ID__C,1,2000) AS VARCHAR(2..."
4,ACTUAL_TIME_LENGTH_MINUTES__C,FLOAT,COLUMN,ACTUAL_TIME_LENGTH_MINUTES__C,ACTUAL_TIME_LENGTH_MINUTES__C,ACTUAL_TIME_LENGTH_MINUTES__C AS ACTUAL_TIME_L...
...,...,...,...,...,...,...
315,FSM_BARRIER_CODE__C,VARCHAR(16777216),COLUMN,FSM_BARRIER_CODE__C,FSM_BARRIER_CODE__C,"CAST(SUBSTR(FSM_BARRIER_CODE__C,1,2000) AS VAR..."
316,FSM_CLIENT_PARTNER_REF__C,VARCHAR(16777216),COLUMN,FSM_CLIENT_PARTNER_REF__C,FSM_CLIENT_PARTNER_REF__C,"CAST(SUBSTR(FSM_CLIENT_PARTNER_REF__C,1,2000) ..."
317,FSM_CLIENT_REF__C,VARCHAR(16777216),COLUMN,FSM_CLIENT_REF__C,FSM_CLIENT_REF__C,"CAST(SUBSTR(FSM_CLIENT_REF__C,1,2000) AS VARCH..."
318,FSM_RESOLVE_CONTRACTUAL_DUE_DATE__C,TIMESTAMP_TZ(9),COLUMN,FSM_RESOLVE_CONTRACTUAL_DUE_DATE__C,RESOLVE_CONTR_DUE_DT_C,FSM_RESOLVE_CONTRACTUAL_DUE_DATE__C AS RESOLVE...


In [182]:
column_list:list = final_df['name_new'].to_list()

In [183]:
columns_str:str = ', '.join(column_list)

In [184]:
final_query:str = f"CREATE OR REPLACE VIEW {db_name}.{schema_name}.{view_name} AS SELECT {columns_str} FROM {db_name}.{schema_name}.{table_for_desc_name};"

In [185]:
final_query

'CREATE OR REPLACE VIEW STITCH.SALESFORCEQASIT.CASE_V AS SELECT CAST(SUBSTR(ACCOUNTID,1,2000) AS VARCHAR(2000)) AS ACCOUNTID, CAST(SUBSTR(ACCOUNT_EXECUTIVE1__C,1,2000) AS VARCHAR(2000)) AS ACCOUNT_EXECUTIVE1__C, CAST(SUBSTR(ACCOUNT_EXECUTIVE__C,1,2000) AS VARCHAR(2000)) AS ACCOUNT_EXECUTIVE__C, CAST(SUBSTR(ACCOUNT_ID__C,1,2000) AS VARCHAR(2000)) AS ACCOUNT_ID__C, ACTUAL_TIME_LENGTH_MINUTES__C AS ACTUAL_TIME_LENGTH_MINUTES__C, CAST(SUBSTR(APPROVAL_WORKFLOW_REQUIRED__C,1,2000) AS VARCHAR(2000)) AS APPROVAL_WORKFLOW_REQUIRED__C, CAST(SUBSTR(BUSINESSHOURSID,1,2000) AS VARCHAR(2000)) AS BUSINESSHOURSID, CAST(SUBSTR(CAC_TEAM_STAMP__C,1,2000) AS VARCHAR(2000)) AS CAC_TEAM_STAMP__C, CAST(SUBSTR(CASENUMBER,1,2000) AS VARCHAR(2000)) AS CASENUMBER, CASE_AGE_BY_BUSINESS_HOURS__C AS CASE_AGE_BY_BUSINESS_HOURS__C, CASE_AGE_IN_BUSINESS_HOURS__C AS CASE_AGE_IN_BUSINESS_HOURS__C, CASE_AGE_IN_MINUTES__C AS CASE_AGE_IN_MINUTES__C, CASE_DURATION__C AS CASE_DURATION__C, CAST(SUBSTR(CASE_OWNER_SUB_CATEGORY_

# Stored Procedure sp_create_salesforce_views

In [144]:
def transform_name(name, type, name_alias):
    if isinstance(type, str) and 'VARCHAR' in type:
        #return name + '0007'
        return f"CAST(SUBSTR({name},1,2000) AS VARCHAR(2000)) AS {name_alias}".format(name, name_alias)
    else:
        return f"{name} AS {name_alias}".format(name, name_alias)

In [145]:
def sp_create_salesforce_views(session: Session, db_name: str, schema_name: str, table_name: str) -> str:
	try:
		# Setting up logger
		logger = logging.getLogger("sp_create_salesforce_views")

		# Getting name of the table
		table_view_mapping = session.sql(f"SELECT * FROM ACCOUNTADMIN_MGMT.UTILITIES.SALESFORCE_TABLE_VIEW_MAPPING WHERE NAME_SALESFORCE_ENVIRONMENT = '{schema_name}' AND NAME_SALESFORCE_TABLE_ORIGINAL = '{table_name}'".format(schema_name, table_name)).collect()
		table_view_mapping_dic = [row.asDict() for row in table_view_mapping][0]
		table_name = table_view_mapping_dic['NAME_SALESFORCE_TABLE_ORIGINAL']
		table_for_desc_name = table_view_mapping_dic['NAME_SNOWFLAKE_RESERVED_WORD']
		view_name = table_view_mapping_dic['NAME_SALESFORCE_VIEW_ALIAS']
		
		# Getting description of the table
		result_describe_table = [row.as_dict() for row in session.sql(f"DESCRIBE TABLE {db_name}.{schema_name}.{table_for_desc_name};".format(db_name, schema_name, table_for_desc_name)).collect()]

		# Removing unnecessary columns
		df_describe_table = pd.DataFrame(result_describe_table)
		df_describe_table.drop(columns=['null?', 'default','primary key', 'unique key',
		'check', 'expression', 'comment', 'policy name'], inplace=True)
		
		# Mixing with Mapping Table
		result_salesforce_mapping = [row.as_dict() for row in session.sql(f"SELECT NAME_SALESFORCE_ATTRIBUTE, NAME_ALIAS FROM ACCOUNTADMIN_MGMT.UTILITIES.SALESFORCE_COLUMN_MAPPING WHERE NAME_SALESFORCE_OBJECT = '{table_name}'".format()).collect()]
		df_salesforce_mapping = pd.DataFrame(result_salesforce_mapping)

		if df_salesforce_mapping.empty:
			final_df = df_describe_table
			final_df['NAME_ALIAS'] = final_df['name']
		else:
			final_df = pd.merge(df_describe_table, df_salesforce_mapping, left_on='name', right_on='NAME_SALESFORCE_ATTRIBUTE', how='left')
			final_df['NAME_ALIAS'] = final_df.apply(lambda x: x['name'] if pd.isna(x['NAME_ALIAS']) else x['NAME_ALIAS'], axis=1)

		# apply the transform_name function to create a new column called 'name_new'
		final_df['name_new'] = final_df.apply(lambda x: transform_name(x['name'], x['type'], x['NAME_ALIAS']), axis=1)

		column_list:list = final_df['name_new'].to_list()

		columns_str:str = ', '.join(column_list)

		# append the final result to something like "CREATE OR REPLACE VIEW {db_name}.{schema_name}}.{table_name}_V AS SELECT {columns_str} FROM {db_name}.{schema_name}.{table_name};"
		final_query:str = f"CREATE OR REPLACE VIEW {db_name}.{schema_name}.{view_name} AS SELECT {columns_str} FROM {db_name}.{schema_name}.{table_for_desc_name};"
		
		session.sql(final_query).collect()

		return "SUCCESS"
	except Exception as e:	
		logger.error(f"Error: {e}", exc_info=True)
		return "FAILURE"

In [168]:
# Getting name of the table
db_name = 'STITCH'
schema_name = 'SALESFORCEQASIT'
table_name:str = 'ASSET'

In [163]:
sp_create_salesforce_views(session, db_name, schema_name, table_name)

'SUCCESS'

## Register sp_create_salesforce_views

In [146]:
session.sproc.register(
	func = sp_create_salesforce_views,
	name = "sp_create_salesforce_views",
	packages = ["snowflake-snowpark-python", "pandas", "numpy"],
	is_permanent = True,
	stage_location = "@ACCOUNTADMIN_MGMT.UTILITIES.ACCOUNTADMIN_MGMT_STAGE",
	source_code_display = False,
	execute_as = 'caller',
	replace=True
)

In [169]:
session.call("sp_create_salesforce_views", db_name, schema_name, table_name)

'SUCCESS'

# All Tables

In [ ]:
#'SALESFORCEQASIT','SALESFORCEFSL3'

In [ ]:
def sp_run_dynamic_salesforce_views(session: Session) -> str:
    database:str = 'STITCH'

    for schema in ['SALESFORCEQASIT']:
        print(f'------->{schema}'.format(schema))
        result = session.sql(f"SHOW TABLES IN STITCH.{schema}".format(schema)).collect()
        df = pd.DataFrame(result)
        tables:list = df['name'].to_list()
        for table in tables:
            if table not in ['_SDC_REJECTED']:
                print(table)
                session.call("sp_create_salesforce_views", database, schema, table)
    
    return "SUCCESS"

In [ ]:
sp_run_dynamic_salesforce_views(session=session)

# Multiple processing

In [ ]:
import multiprocessing as mp

def process_table(database, schema, table):
    if table not in ['_SDC_REJECTED']:
        print(table)
        session.call("sp_create_salesforce_views", database, schema, table)

def sp_run_dynamic_salesforce_views(session: Session) -> str:
    database:str = 'STITCH'

    for schema in ['SALESFORCEQASIT']:
        print(f'------->{schema}'.format(schema))
        result = session.sql(f"SHOW TABLES IN STITCH.{schema}".format(schema)).collect()
        df = pd.DataFrame(result)
        tables:list = df['name'].to_list()
        pool = mp.Pool(processes=4)
        for table in tables:
            pool.apply_async(process_table, args=(database, schema, table))
        pool.close()
        pool.join()
    
    return "SUCCESS"

In [ ]:
sp_run_dynamic_salesforce_views(session=session)

# Creating stored procedure

In [ ]:
session.sproc.register(
	func=sp_run_dynamic_salesforce_views,
	name="sp_run_dynamic_salesforce_views",
	packages=["snowflake-snowpark-python", "pandas", "numpy"],
	is_permanent=True,
	stage_location="@ACCOUNTADMIN_MGMT.UTILITIES.ACCOUNTADMIN_MGMT_STAGE",
	source_code_display = False,
	execute_as = 'caller',
	replace=True
)

In [ ]:
session.call("sp_run_dynamic_salesforce_views")

## Creating Task

In [4]:
session.use_role("SYSADMIN")
session.use_database("ACCOUNTADMIN_MGMT")
session.use_warehouse("ACCOUNTADMIN_MGMT")
session.use_schema("UTILITIES")

In [9]:
task_run_dynamic_salesforce_views = """
CREATE OR REPLACE TASK ACCOUNTADMIN_MGMT.UTILITIES.TASK_RUN_DYNAMIC_SALESFORCE_VIEWS COMMENT = 'Run dynamic salesforce views'
    WAREHOUSE = 'ACCOUNTADMIN_MGMT'
    ALLOW_OVERLAPPING_EXECUTION = FALSE
	AFTER ACCOUNTADMIN_MGMT.UTILITIES.TASK_RUN_DYNAMIC_SALESFORCE_COLUMN_DETECTOR

AS
        CALL ACCOUNTADMIN_MGMT.UTILITIES.SP_RUN_DYNAMIC_SALESFORCE_VIEWS();
"""

In [10]:
session.sql(task_run_dynamic_salesforce_views).collect()

[Row(status='Task TASK_RUN_DYNAMIC_SALESFORCE_VIEWS successfully created.')]

In [12]:
session.sql("ALTER TASK ACCOUNTADMIN_MGMT.UTILITIES.TASK_RUN_DYNAMIC_SALESFORCE_VIEWS RESUME;").collect()

[Row(status='Statement executed successfully.')]

In [ ]:
session.sql("EXECUTE TASK ACCOUNTADMIN_MGMT.UTILITIES.TASK_RUN_DYNAMIC_SALESFORCE_VIEWS;").collect()

In [ ]:
session.close()